<a href="https://colab.research.google.com/github/EricCallaway/COSC_5313_TensorFlow_CNN_model/blob/main/CNN_model_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load TensorFlow and CIFAR10 dataset Libriaries

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

Use load_data() function to retrieve the traning and test images.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Check the size of the training images x_train classes lables y_train

In [ ]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


Normalize pixels from 0-255 to 0-1

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

Define CNN Model

Below is the Sequential method. This is by far the easiest way to create a CNN. There is very little flexibility with the sequential method.

In [ ]:
from tensorflow.keras.models import Sequential 
sequential_model = tf.keras.Sequential([
                          
                          tf.keras.Input(shape=(32,32,3)),
                          tf.keras.layers.Conv2D(32,3,padding='same', activation='relu', name="First_CNN_Layer"),
                          tf.keras.layers.MaxPooling2D((2,2), name="First_MaxPool_Layer"),
                          tf.keras.layers.Conv2D(64,3,padding='same', activation='relu',name="Second_CNN_Layer"),
                          tf.keras.layers.MaxPooling2D((2,2),name="Second_MaxPool_Layer"),
                          tf.keras.layers.Conv2D(128,3,padding='same', activation='relu',name="Third_CNN_Layer"),
                          tf.keras.layers.MaxPooling2D((2,2),name="Third_MaxPool_Layer"),
                          tf.keras.layers.Flatten(),
                          tf.keras.layers.Dense(64, activation='relu', name="First_Dense_Layer"),
                          tf.keras.layers.Dense(10, activation='softmax', name="Second_Dense_Layer")

])
sequential_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 First_CNN_Layer (Conv2D)    (None, 32, 32, 32)        896       
                                                                 
 First_MaxPool_Layer (MaxPoo  (None, 16, 16, 32)       0         
 ling2D)                                                         
                                                                 
 Second_CNN_Layer (Conv2D)   (None, 16, 16, 64)        18496     
                                                                 
 Second_MaxPool_Layer (MaxPo  (None, 8, 8, 64)         0         
 oling2D)                                                        
                                                                 
 Third_CNN_Layer (Conv2D)    (None, 8, 8, 128)         73856     
                                                                 
 Third_MaxPool_Layer (MaxPoo  (None, 4, 4, 128)       

Functional API Method:
This method is very useful for debuggin purposes for a very large network where you need debugging at layers level. This method enables us to provide multiple inputs and take multiple outputs as well from the CNN model which is not possible using the Sequential method. 

In [ ]:
inputs = tf.keras.Input(shape=(32,32,3), name="Input_Layer")
x = tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='relu',name="First_CNN_Layer")(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2), name='First_MaxPool_Layer')(x)
x = tf.keras.layers.Conv2D(64,(3,3), padding='same', activation='relu', name='Second_CNN_Layer')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),  name='Second_MaxPool_Layer' )(x)
x = tf.keras.layers.Conv2D(128,(3,3), padding='same', activation='relu', name='Third_CNN_Layer')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),  name='Third_MaxPool_Layer' )(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu',name='First_Dense_Layer')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax',name='Second_Dense_Layer')(x)

functional_api_model = tf.keras.Model( inputs = inputs, outputs = outputs)
functional_api_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (InputLayer)    [(None, 32, 32, 3)]       0         
                                                                 
 First_CNN_Layer (Conv2D)    (None, 32, 32, 32)        896       
                                                                 
 First_MaxPool_Layer (MaxPoo  (None, 16, 16, 32)       0         
 ling2D)                                                         
                                                                 
 Second_CNN_Layer (Conv2D)   (None, 16, 16, 64)        18496     
                                                                 
 Second_MaxPool_Layer (MaxPo  (None, 8, 8, 64)         0         
 oling2D)                                                        
                                                                 
 Third_CNN_Layer (Conv2D)    (None, 8, 8, 128)         7385

The overall idea of the Functional API method is to pass a layer to the next layer as functional input. Inputs and outputs are defined explicityly and then pass to a keras.Model class to define a model. This way you can pass multiple inputs and outputs as well to the keras.Model class. 

Model Sublcassing: 
This is a complex but most flexible method to define a model in TensorFlow. We define a model class by inherting keras.Model class and wrap everything within the class. 

In [ ]:
class CustomModel(tf.keras.Model):

	def __init__(self):
		
		super(CustomModel,self).__init__();
		
		self.first_cnn_layer = tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='relu',name="First_CNN_Layer")
		self.first_pooling = tf.keras.layers.MaxPooling2D((2,2), name="First_MaxPool_Layer")
		self.second_cnn_layer = tf.keras.layers.Conv2D(64,(3,3),padding='same', activation='relu',name="Second_CNN_Layer")
		self.second_pooling = tf.keras.layers.MaxPooling2D((2,2), name="Second_MaxPool_Layer")
		self.third_cnn_layer = tf.keras.layers.Conv2D(128,(3,3), padding='same',activation='relu',name="Third_CNN_Layer")
		self.third_pooling = tf.keras.layers.MaxPooling2D((2,2), name="Third_MaxPool_Layer")
		self.flatten = tf.keras.layers.Flatten()
		self.first_dense_layer = tf.keras.layers.Dense(64, activation='relu', name="First_Dense_Layer")
		self.second_dense_layer = tf.keras.layers.Dense(10, name="Second_Dense_Layer")

	def call(self, input_tensor, training = False):

		x = self.first_cnn_layer(input_tensor, training = training)
		x = self.first_pooling(x)
		x = self.second_cnn_layer(x, training = training)
		x = self.second_pooling(x)
		x = self.third_cnn_layer(x, training = training) 
		x = self.third_pooling(x)
		x = self.flatten(x)
		x = self.first_dense_layer(x, training = training)
		x = self.second_dense_layer(x, training = training)

		return x

The overall idea is to first define all layers within the constructor function of the model calss and then define a call method to call these layers. Training parameter in the call method will be automatically set to TRUE by the fit() method and FALSE by the evaluate() method.

Train the Model

In [ ]:
sequential_model.compile(
	loss = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True ),
	optimizer = tf.keras.optimizers.Adam(lr = .001),
	metrics = ['accuracy']
	)

sequential_model.fit(x_train, y_train, epochs= 10, verbose=2, batch_size = 64)
sequential_model.evaluate(x_test, y_test, verbose = 2, batch_size=64)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


782/782 - 105s - loss: 1.5304 - accuracy: 0.4452 - 105s/epoch - 134ms/step
Epoch 2/10
782/782 - 106s - loss: 1.0972 - accuracy: 0.6114 - 106s/epoch - 135ms/step
Epoch 3/10
782/782 - 103s - loss: 0.9219 - accuracy: 0.6769 - 103s/epoch - 132ms/step
Epoch 4/10
782/782 - 103s - loss: 0.8148 - accuracy: 0.7170 - 103s/epoch - 132ms/step
Epoch 5/10
782/782 - 104s - loss: 0.7262 - accuracy: 0.7477 - 104s/epoch - 133ms/step
Epoch 6/10
782/782 - 104s - loss: 0.6530 - accuracy: 0.7719 - 104s/epoch - 132ms/step
Epoch 7/10
782/782 - 103s - loss: 0.6003 - accuracy: 0.7906 - 103s/epoch - 132ms/step
Epoch 8/10
782/782 - 104s - loss: 0.5453 - accuracy: 0.8089 - 104s/epoch - 134ms/step
Epoch 9/10
782/782 - 104s - loss: 0.4918 - accuracy: 0.8281 - 104s/epoch - 134ms/step
Epoch 10/10
782/782 - 105s - loss: 0.4462 - accuracy: 0.8411 - 105s/epoch - 134ms/step
157/157 - 6s - loss: 0.8600 - accuracy: 0.7301 - 6s/epoch - 36ms/step


[0.8600209951400757, 0.7300999760627747]